# LLM Market Analytics Examples

This notebook demonstrates how to use the LLM-based market forecaster for analyzing and predicting market movements.

## Load and Prepare Sample Data

Let's create some sample market data for demonstration purposes.

In [1]:
from quantlib.datasource.service import DataService
data_service = DataService.create_default_service()

# Define parameters for data fetching
symbol = "IBOV"  # Change to your desired symbol
start_date = "2021-01-01"  # Adjust as needed
end_date = "2025-05-16"    # Adjust as needed
interval = "1m"            # 1-minute data

ModuleNotFoundError: No module named 'MetaTrader5'

In [10]:
ts_object = data_service.get_close_prices(
        symbol=symbol,
        source='mt5',
        start_date=start_date,
        end_date=end_date,
        interval=interval
    )

In [15]:
ts_object.data.to_parquet('petr_returns.parquet')

## Use the data and build on unix enviroment

In [2]:
import pandas as pd
import numpy as np
import torch
from quantlib.analytics.llm import LLMForecaster, MarketTokenizer
from quantlib.data.timeseries import TimeSeries

In [3]:
ts = TimeSeries(pd.read_parquet('quantlib/notebooks/win_returns.parquet'))

In [4]:
ret = ts.returns(intraday_only=True)
returns = ret.data[(ret.data.index.hour >= 10) & (ret.data.index.hour < 17)]

In [5]:
# 2. Initialize the forecaster
forecaster = LLMForecaster()


In [6]:
# 3. Prepare the dataset
forecaster.build_dataset(returns['close'])

In [7]:
# 4. Build and train the model
forecaster.build_model()
forecaster.train()

Epoch 1, Loss: 0.2077
Epoch 2, Loss: 0.0354
Epoch 3, Loss: 0.0344
Epoch 4, Loss: 0.0341
Epoch 5, Loss: 0.0339


In [8]:
# 5. Generate predictions
# Create a prompt from recent market data
recent_data = returns['close'].iloc[-60:].values
prompt = torch.tensor(
    MarketTokenizer.series_to_tokens(
        pd.Series(recent_data),
        forecaster.vocab_size
    )[0],
    dtype=torch.long
).unsqueeze(0)  # Add batch dimension

# Generate predictions
predicted_tokens = forecaster.fit(prompt, steps=5)  # Predict next 5 days

# Convert predictions back to returns
predicted_returns = MarketTokenizer.tokens_to_values(
    np.array(predicted_tokens),
    forecaster.bins
)

print("Predicted returns for next periods:", predicted_returns[:-5])

Predicted returns for next periods: [ 0.00267595  0.00792016  0.00729085 -0.00508548  0.00792016 -0.01326645
  0.00330525  0.0049834   0.00414432 -0.00109988  0.0056127  -0.00193896
 -0.00109988  0.00330525 -0.00571479 -0.00109988 -0.01326645 -0.00277803
  0.00036849  0.0049834  -0.00571479  0.0009978  -0.00193896  0.00729085
 -0.00193896  0.0049834   0.01022761  0.01022761 -0.00047058 -0.00424641
 -0.00109988  0.00414432 -0.00340734 -0.00277803  0.00729085  0.0056127
  0.00267595 -0.00047058 -0.00508548 -0.00802224  0.00267595  0.01106668
 -0.00109988 -0.00047058 -0.00277803 -0.00193896  0.0129546   0.0009978
 -0.00109988 -0.00109988 -0.00949062 -0.00193896  0.0129546   0.00330525
  0.00036849  0.00183687]


In [9]:
# 6. Evaluate predictions (example with simple metrics)
actual_returns = returns.iloc[-5:].values
mse = np.mean((predicted_returns - actual_returns) ** 2)
print(f"Mean Squared Error: {mse:.6f}")

Mean Squared Error: 0.000031
